In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# # 创建随机森林分类器
# rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=None)

# # 创建 Boruta 实例，降低 perc 参数以增加选择的特征数量
# boruta_selector = BorutaPy(
#     estimator=rf,
#     n_estimators='auto',
#     perc=80,  # 将阈值降低到 80%
#     verbose=2,
#     random_state=42
# )

In [3]:
Clinical = pd.read_csv('../../DATA/BRCA/Human__TCGA_BRCA__MS__Clinical__Clinical__01_28_2016__BI__Clinical__Firehose_common_samples.tsv', sep='\t')
# 或者直接在原数据框中进行删除
# Clinical.dropna(subset=[Clinical.columns[-2]], inplace=True)
Clinical

,Unnamed: 0,years_to_birth,Tumor_purity,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,histological_type,number_of_lymph_nodes,PAM50,...,PR.Status,HER2.Status,gender,radiation_therapy,race,ethnicity,Median_overall_survival,overall_survival,status,overallsurvival
0,TCGA.D8.A1JH,56.0,0.6126,stagei,t1,n0,m0,infiltratingductalcarcinoma,NaN,LumA,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,426.0,0.0,"426,0"
1,TCGA.OL.A5RW,40.0,0.8121,stageii,t1,n1,NaN,infiltratingductalcarcinoma,1.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,NaN,0.0,1106.0,0.0,"1106,0"
2,TCGA.XX.A899,46.0,0.6069,stageiii,t1,n2,NaN,infiltratinglobularcarcinoma,5.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,467.0,0.0,"467,0"
3,TCGA.S3.A6ZH,29.0,0.8698,stageiii,t3,n3,NaN,infiltratingductalcarcinoma,14.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,641.0,0.0,"641,0"
4,TCGA.AQ.A54N,51.0,0.9075,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,Basal,...,NaN,NaN,female,no,blackorafricanamerican,hispanicorlatino,0.0,78.0,0.0,"78,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,TCGA.GM.A5PX,65.0,0.7913,stageii,t2,n1,m0,infiltratinglobularcarcinoma,1.0,NaN,...,NaN,NaN,female,no,white,nothispanicorlatino,0.0,551.0,0.0,"551,0"
766,TCGA.AC.A5XU,74.0,0.7798,stageii,t3,n0,NaN,infiltratingductalcarcinoma,0.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,455.0,0.0,"455,0"
767,TCGA.AR.A1AN,46.0,0.6708,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,NaN,female,yes,white,hispanicorlatino,0.0,2920.0,0.0,"2920,0"
768,TCGA.E9.A1RG,62.0,0.7904,stageiii,t1,n2,m0,infiltratingductalcarcinoma,4.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,647.0,0.0,"647,0"


In [4]:
# 读取制表符分隔的CSV文件
Methylation = pd.read_csv('../../DATA/BRCA/BRCA_Methylation.csv')
# 显示 DataFrame
Methylation

,PROX1,SH2B3,RSPO3,HOXD3,HMX2,SHOX2,GJB6,MAPK1,PDGFRA,TEAD3,...,NOP56,PIH1D2,EXOSC1,REXO4,TRMT112,PRTFDC1,PRDM2,MTFMT,EIF1AD,FCGRT
0,-0.3947,0.3193,0.1971,-0.0426,-0.1288,-0.1590,-0.2084,-0.4558,-0.2251,-0.2197,...,-0.0756,-0.4602,-0.4225,-0.0336,-0.3139,-0.4096,-0.4102,-0.4736,-0.4522,-0.1503
1,-0.4754,0.3408,0.0615,0.1237,-0.4644,-0.0780,-0.4734,-0.4738,-0.3987,-0.3110,...,-0.3802,-0.4293,-0.4798,-0.0455,-0.3757,-0.4630,-0.4789,-0.4707,-0.4813,-0.3234
2,-0.0863,0.3551,0.1403,0.1394,-0.2340,-0.0532,-0.1977,-0.4763,-0.0906,-0.2305,...,-0.3447,-0.3862,-0.4814,0.0116,-0.3964,-0.3886,-0.4801,-0.4754,-0.4828,-0.0965
3,-0.2583,0.3964,0.0686,0.3516,-0.4030,0.0850,-0.3241,-0.4781,0.1752,-0.3810,...,-0.4279,-0.3667,-0.4777,0.1632,-0.4000,-0.4271,-0.4775,-0.4725,-0.4843,-0.3937
4,-0.4699,0.3242,-0.2212,0.3626,-0.4097,0.3497,-0.4797,-0.4764,-0.3897,-0.3840,...,-0.3979,-0.4168,-0.4850,0.0333,-0.3639,-0.4688,-0.4766,-0.4649,-0.4830,-0.3935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.2350,0.3391,0.1727,0.2844,-0.0159,0.0174,0.0221,-0.4761,-0.1010,-0.2657,...,-0.1847,-0.4424,-0.4846,-0.0152,-0.3879,0.0175,-0.4792,-0.4734,-0.4823,-0.0147
766,-0.0497,0.3121,0.3168,0.3675,0.0914,0.1603,-0.0460,-0.4770,0.0527,-0.2806,...,-0.4468,-0.4221,-0.4839,-0.0538,-0.3578,-0.4032,-0.4811,-0.4659,-0.4825,0.1493
767,0.0042,0.3174,0.2500,0.3374,-0.4054,0.0424,-0.4743,-0.4672,-0.0599,-0.3227,...,-0.3062,-0.4441,-0.4820,-0.0172,-0.3558,-0.0246,-0.4811,-0.4745,-0.4823,0.0561
768,-0.2187,0.3242,0.3495,0.3104,-0.2608,-0.3840,-0.4116,-0.4801,0.0197,-0.3545,...,0.0776,-0.4326,-0.4855,0.0375,-0.3625,-0.0044,-0.4801,-0.4746,-0.4842,-0.2489


In [5]:
# 读取制表符分隔的CSV文件
SCNA = pd.read_csv('../../DATA/BRCA/BRCA_SCNA.csv')
# 显示 DataFrame
SCNA

,PRKCZ,RPL22,TNFRSF9,PIK3CD,CTNNBIP1,MTOR,PLA2G2D,WNT4,RUNX3,ZNF683,...,TOPAZ1,AKR7A3,HYI,GRHPR,MSTO1,ARMC1,CLUH,GPR101,SMR3A,SMR3B
0,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,...,-0.018,-0.001,-0.001,-0.008,-0.001,0.1920,0.002,0.0300,0.1170,0.1170
1,0.304,0.363,0.333,0.338,0.338,0.338,0.338,0.338,0.262,0.262,...,-0.629,0.338,0.310,0.258,1.207,-0.0180,-0.709,0.7000,-0.7490,-0.7490
2,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,...,0.001,-0.071,-0.071,-0.069,0.275,0.1520,0.008,0.0010,-0.0062,-0.0062
3,-0.423,-0.423,-0.423,-0.423,-0.423,-0.423,-0.427,-0.427,-0.387,-0.387,...,-0.753,-0.427,-0.387,-0.312,1.199,0.1170,-0.837,-0.1500,-0.2660,-0.2660
4,0.100,0.100,0.100,0.100,0.100,0.100,0.108,0.083,0.709,0.033,...,-0.604,0.108,0.065,-0.593,1.086,0.6550,0.630,0.2420,0.0330,0.0330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,...,0.004,-0.013,-0.016,-0.002,1.034,0.0074,-0.020,0.0010,0.0040,0.0040
766,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,...,0.255,-0.525,-0.525,-0.526,0.638,-0.5190,-0.257,0.0358,-0.0050,-0.0050
767,0.053,0.053,0.053,0.053,0.053,0.053,0.053,0.053,0.053,0.053,...,0.012,0.053,0.019,0.011,0.441,-0.0100,-0.449,-0.0600,1.2350,1.2350
768,-0.373,-0.373,-0.373,-0.373,-0.373,-0.373,-0.373,-0.365,-0.365,-0.365,...,-0.385,-0.373,-0.370,-0.384,0.502,0.5370,-0.372,0.3820,0.0660,0.0660


In [6]:
# 读取制表符分隔的CSV文件
RNAseq = pd.read_csv('../../DATA/BRCA/BRCA_RNAseq.csv')
# 显示 DataFrame
RNAseq

,ABL1,ACADM,ACAT1,ACO2,ADA,AKT1,AKT2,ALDH1A2,ALDH1A3,ALOX15B,...,SURF1,UQCR10,UQCR11,UQCRB,UQCRC2,UQCRFS1,UQCRHL,UQCRH,WDR93,CGB7
0,11.1922,9.8565,10.6013,11.0607,6.9935,11.8391,11.0237,5.5665,9.8314,8.3599,...,9.3959,9.3948,9.5402,11.0453,11.5041,10.3084,7.2733,9.3964,5.4703,1.01410
1,9.7755,8.5877,9.6203,11.5186,7.9402,12.8907,11.4956,4.8889,8.2605,5.8519,...,9.9592,11.0420,10.9705,12.3146,11.3483,11.9523,10.7069,12.8206,3.4653,5.16440
2,11.3155,10.4236,9.8686,11.3374,7.7755,11.6033,11.6151,6.9614,9.1619,12.6663,...,9.3351,9.8759,10.1307,11.3989,11.6124,10.0885,8.3823,10.2502,4.5986,2.39290
3,10.6537,11.4840,8.8371,11.3729,5.5148,12.2656,12.0471,5.8103,6.1662,3.7653,...,9.8281,10.5420,10.3690,11.6772,12.0496,11.1781,8.7979,10.6588,3.5877,0.39500
4,10.2955,9.0557,9.3345,10.4181,8.6322,12.0699,11.4128,1.9008,9.5649,4.5994,...,9.3037,10.5910,10.1829,12.3978,10.3072,9.6428,9.7032,11.9100,4.0035,0.75150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,10.4632,10.0001,11.4173,10.7544,7.8649,11.6967,10.9855,5.8199,7.9579,9.4490,...,9.9846,10.9903,10.8845,11.8077,11.8706,10.5575,8.4381,10.8952,6.7766,1.56260
766,11.6356,8.6519,10.7598,11.9914,7.5397,12.1868,12.8083,3.1336,8.6952,5.0729,...,9.7506,10.5215,10.0663,10.9821,11.9018,9.7408,7.7471,10.0362,5.9552,1.11530
767,11.4497,10.7140,10.3352,11.2001,6.6691,12.7403,11.6774,7.7411,8.1145,5.0381,...,9.3969,9.6158,10.3063,11.4330,11.1893,10.5109,9.0697,11.0288,3.6354,1.43788
768,10.4021,9.2604,10.6261,11.6921,8.7099,13.1232,11.0644,3.5396,6.0385,6.8356,...,10.5980,11.4647,11.2425,12.1177,12.2192,11.1764,9.0552,10.9535,5.6958,3.63070


In [7]:
# 找到Clinical中包含NaN值的行的索引
nan_indices = Clinical[Clinical.iloc[:, -2].isna()].index

In [8]:
# 删除Clinical中包含NaN值的行
Clinical.dropna(subset=[Clinical.columns[-2]], inplace=True)
Clinical

,Unnamed: 0,years_to_birth,Tumor_purity,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,histological_type,number_of_lymph_nodes,PAM50,...,PR.Status,HER2.Status,gender,radiation_therapy,race,ethnicity,Median_overall_survival,overall_survival,status,overallsurvival
0,TCGA.D8.A1JH,56.0,0.6126,stagei,t1,n0,m0,infiltratingductalcarcinoma,NaN,LumA,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,426.0,0.0,"426,0"
1,TCGA.OL.A5RW,40.0,0.8121,stageii,t1,n1,NaN,infiltratingductalcarcinoma,1.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,NaN,0.0,1106.0,0.0,"1106,0"
2,TCGA.XX.A899,46.0,0.6069,stageiii,t1,n2,NaN,infiltratinglobularcarcinoma,5.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,467.0,0.0,"467,0"
3,TCGA.S3.A6ZH,29.0,0.8698,stageiii,t3,n3,NaN,infiltratingductalcarcinoma,14.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,641.0,0.0,"641,0"
4,TCGA.AQ.A54N,51.0,0.9075,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,Basal,...,NaN,NaN,female,no,blackorafricanamerican,hispanicorlatino,0.0,78.0,0.0,"78,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,TCGA.GM.A5PX,65.0,0.7913,stageii,t2,n1,m0,infiltratinglobularcarcinoma,1.0,NaN,...,NaN,NaN,female,no,white,nothispanicorlatino,0.0,551.0,0.0,"551,0"
766,TCGA.AC.A5XU,74.0,0.7798,stageii,t3,n0,NaN,infiltratingductalcarcinoma,0.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,455.0,0.0,"455,0"
767,TCGA.AR.A1AN,46.0,0.6708,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,NaN,female,yes,white,hispanicorlatino,0.0,2920.0,0.0,"2920,0"
768,TCGA.E9.A1RG,62.0,0.7904,stageiii,t1,n2,m0,infiltratingductalcarcinoma,4.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,647.0,0.0,"647,0"


In [9]:
# 如果需要在原DataFrame中删除这些行
Methylation.drop(index=nan_indices, inplace=True)
Methylation

,PROX1,SH2B3,RSPO3,HOXD3,HMX2,SHOX2,GJB6,MAPK1,PDGFRA,TEAD3,...,NOP56,PIH1D2,EXOSC1,REXO4,TRMT112,PRTFDC1,PRDM2,MTFMT,EIF1AD,FCGRT
0,-0.3947,0.3193,0.1971,-0.0426,-0.1288,-0.1590,-0.2084,-0.4558,-0.2251,-0.2197,...,-0.0756,-0.4602,-0.4225,-0.0336,-0.3139,-0.4096,-0.4102,-0.4736,-0.4522,-0.1503
1,-0.4754,0.3408,0.0615,0.1237,-0.4644,-0.0780,-0.4734,-0.4738,-0.3987,-0.3110,...,-0.3802,-0.4293,-0.4798,-0.0455,-0.3757,-0.4630,-0.4789,-0.4707,-0.4813,-0.3234
2,-0.0863,0.3551,0.1403,0.1394,-0.2340,-0.0532,-0.1977,-0.4763,-0.0906,-0.2305,...,-0.3447,-0.3862,-0.4814,0.0116,-0.3964,-0.3886,-0.4801,-0.4754,-0.4828,-0.0965
3,-0.2583,0.3964,0.0686,0.3516,-0.4030,0.0850,-0.3241,-0.4781,0.1752,-0.3810,...,-0.4279,-0.3667,-0.4777,0.1632,-0.4000,-0.4271,-0.4775,-0.4725,-0.4843,-0.3937
4,-0.4699,0.3242,-0.2212,0.3626,-0.4097,0.3497,-0.4797,-0.4764,-0.3897,-0.3840,...,-0.3979,-0.4168,-0.4850,0.0333,-0.3639,-0.4688,-0.4766,-0.4649,-0.4830,-0.3935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.2350,0.3391,0.1727,0.2844,-0.0159,0.0174,0.0221,-0.4761,-0.1010,-0.2657,...,-0.1847,-0.4424,-0.4846,-0.0152,-0.3879,0.0175,-0.4792,-0.4734,-0.4823,-0.0147
766,-0.0497,0.3121,0.3168,0.3675,0.0914,0.1603,-0.0460,-0.4770,0.0527,-0.2806,...,-0.4468,-0.4221,-0.4839,-0.0538,-0.3578,-0.4032,-0.4811,-0.4659,-0.4825,0.1493
767,0.0042,0.3174,0.2500,0.3374,-0.4054,0.0424,-0.4743,-0.4672,-0.0599,-0.3227,...,-0.3062,-0.4441,-0.4820,-0.0172,-0.3558,-0.0246,-0.4811,-0.4745,-0.4823,0.0561
768,-0.2187,0.3242,0.3495,0.3104,-0.2608,-0.3840,-0.4116,-0.4801,0.0197,-0.3545,...,0.0776,-0.4326,-0.4855,0.0375,-0.3625,-0.0044,-0.4801,-0.4746,-0.4842,-0.2489


In [10]:
# 如果需要在原DataFrame中删除这些行
SCNA.drop(index=nan_indices, inplace=True)
SCNA

,PRKCZ,RPL22,TNFRSF9,PIK3CD,CTNNBIP1,MTOR,PLA2G2D,WNT4,RUNX3,ZNF683,...,TOPAZ1,AKR7A3,HYI,GRHPR,MSTO1,ARMC1,CLUH,GPR101,SMR3A,SMR3B
0,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,-0.001,...,-0.018,-0.001,-0.001,-0.008,-0.001,0.1920,0.002,0.0300,0.1170,0.1170
1,0.304,0.363,0.333,0.338,0.338,0.338,0.338,0.338,0.262,0.262,...,-0.629,0.338,0.310,0.258,1.207,-0.0180,-0.709,0.7000,-0.7490,-0.7490
2,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,-0.071,...,0.001,-0.071,-0.071,-0.069,0.275,0.1520,0.008,0.0010,-0.0062,-0.0062
3,-0.423,-0.423,-0.423,-0.423,-0.423,-0.423,-0.427,-0.427,-0.387,-0.387,...,-0.753,-0.427,-0.387,-0.312,1.199,0.1170,-0.837,-0.1500,-0.2660,-0.2660
4,0.100,0.100,0.100,0.100,0.100,0.100,0.108,0.083,0.709,0.033,...,-0.604,0.108,0.065,-0.593,1.086,0.6550,0.630,0.2420,0.0330,0.0330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,...,0.004,-0.013,-0.016,-0.002,1.034,0.0074,-0.020,0.0010,0.0040,0.0040
766,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,-0.525,...,0.255,-0.525,-0.525,-0.526,0.638,-0.5190,-0.257,0.0358,-0.0050,-0.0050
767,0.053,0.053,0.053,0.053,0.053,0.053,0.053,0.053,0.053,0.053,...,0.012,0.053,0.019,0.011,0.441,-0.0100,-0.449,-0.0600,1.2350,1.2350
768,-0.373,-0.373,-0.373,-0.373,-0.373,-0.373,-0.373,-0.365,-0.365,-0.365,...,-0.385,-0.373,-0.370,-0.384,0.502,0.5370,-0.372,0.3820,0.0660,0.0660


In [11]:
# 如果需要在原DataFrame中删除这些行
RNAseq.drop(index=nan_indices, inplace=True)
RNAseq

,ABL1,ACADM,ACAT1,ACO2,ADA,AKT1,AKT2,ALDH1A2,ALDH1A3,ALOX15B,...,SURF1,UQCR10,UQCR11,UQCRB,UQCRC2,UQCRFS1,UQCRHL,UQCRH,WDR93,CGB7
0,11.1922,9.8565,10.6013,11.0607,6.9935,11.8391,11.0237,5.5665,9.8314,8.3599,...,9.3959,9.3948,9.5402,11.0453,11.5041,10.3084,7.2733,9.3964,5.4703,1.01410
1,9.7755,8.5877,9.6203,11.5186,7.9402,12.8907,11.4956,4.8889,8.2605,5.8519,...,9.9592,11.0420,10.9705,12.3146,11.3483,11.9523,10.7069,12.8206,3.4653,5.16440
2,11.3155,10.4236,9.8686,11.3374,7.7755,11.6033,11.6151,6.9614,9.1619,12.6663,...,9.3351,9.8759,10.1307,11.3989,11.6124,10.0885,8.3823,10.2502,4.5986,2.39290
3,10.6537,11.4840,8.8371,11.3729,5.5148,12.2656,12.0471,5.8103,6.1662,3.7653,...,9.8281,10.5420,10.3690,11.6772,12.0496,11.1781,8.7979,10.6588,3.5877,0.39500
4,10.2955,9.0557,9.3345,10.4181,8.6322,12.0699,11.4128,1.9008,9.5649,4.5994,...,9.3037,10.5910,10.1829,12.3978,10.3072,9.6428,9.7032,11.9100,4.0035,0.75150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,10.4632,10.0001,11.4173,10.7544,7.8649,11.6967,10.9855,5.8199,7.9579,9.4490,...,9.9846,10.9903,10.8845,11.8077,11.8706,10.5575,8.4381,10.8952,6.7766,1.56260
766,11.6356,8.6519,10.7598,11.9914,7.5397,12.1868,12.8083,3.1336,8.6952,5.0729,...,9.7506,10.5215,10.0663,10.9821,11.9018,9.7408,7.7471,10.0362,5.9552,1.11530
767,11.4497,10.7140,10.3352,11.2001,6.6691,12.7403,11.6774,7.7411,8.1145,5.0381,...,9.3969,9.6158,10.3063,11.4330,11.1893,10.5109,9.0697,11.0288,3.6354,1.43788
768,10.4021,9.2604,10.6261,11.6921,8.7099,13.1232,11.0644,3.5396,6.0385,6.8356,...,10.5980,11.4647,11.2425,12.1177,12.2192,11.1764,9.0552,10.9535,5.6958,3.63070


In [12]:
# 定义特征和响应变量
X = Methylation
y = Clinical['status']

In [13]:
y

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
765    0.0
766    0.0
767    0.0
768    0.0
769    0.0
Name: status, Length: 755, dtype: float64

In [14]:
# 分割数据集：80% 用于训练，20% 用于测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建随机森林分类器
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=None)

# 创建 Boruta 特征选择器
boruta_selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',  # 自动选择随机森林的树的数量
    perc=80,  # 设置80%阈值，选择比80%分位数更重要的特征
    verbose=2,  # 输出详细信息
    random_state=42
)

# 训练 Boruta 模型
boruta_selector.fit(X_train.values, y_train.values)

# 获取被认为是重要的特征
important_features = X.columns[boruta_selector.support_]

# 获取可能重要的特征
tentative_features = X.columns[boruta_selector.support_weak_]

# 合并重要特征和可能重要的特征
selected_features = list(important_features) + list(tentative_features)

# 创建一个新的数据框，包含所有被选中的特征和目标变量
X_selected = X[selected_features]

# 创建 StandardScaler 对象
scaler = StandardScaler()

# 对选择的特征进行 Z 得分归一化
X_selected_scaled = scaler.fit_transform(X_selected)

# 将归一化后的数据转换回 DataFrame，并保留原有的列名
Methylation = pd.DataFrame(X_selected_scaled, columns=X_selected.columns)

# 打印归一化后的特征数据框
print("Z 得分归一化后的特征数据框:")
Methylation

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	12049
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	7079
Rejected: 	4970
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	7079
Rejected: 	4970
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	7079
Rejected: 	4970
Iteration: 	11 / 100
Confirmed: 	104
Tentative: 	6975
Rejected: 	4970
Iteration: 	12 / 100
Confirmed: 	104
Tentative: 	4723
Rejected: 	7222
Iteration: 	13 / 100
Confirmed: 	171
Tentative: 	4656
Rejected: 	7222
Iteration: 	14 / 100
Confirmed: 	171
Tentative: 	4656
Rejected: 	7222
Iteration: 	15 / 100
Confirmed: 	171
Tentative: 	4656
Rejected:

,HMX2,ATP6V1B1,EN2,FGFR2,PTK7,FOXF1,LEF1,NEUROD1,TCF21,TTC39C,...,KLK1,BRPF1,PCP4,FAM168A,RPS28,CLVS2,PAICS,CDKL1,KRI1,EBNA1BP2
0,0.318493,-0.720555,-1.047440,-0.313577,0.149325,-0.286816,-0.421182,-1.425711,2.860720,-0.572211,...,0.827191,-0.229341,-0.400396,5.339060,-0.577031,0.391534,2.033591,0.166240,0.423784,25.644377
1,-1.330506,-4.590240,-1.036738,-1.457112,-1.201924,1.321021,2.381067,-1.018723,0.578405,0.543191,...,-0.713459,-1.429976,-0.437187,0.724184,-0.155813,-1.185081,1.793079,-3.249565,-1.211709,0.208847
2,-0.198416,0.173972,0.557359,-0.049035,-0.326864,-0.333053,-0.595506,-0.123930,-0.678734,0.170988,...,0.434844,1.185692,-0.168827,0.179733,-0.366422,-0.209358,0.542412,0.607785,0.238875,0.241828
3,-1.028812,-3.396937,2.186557,0.865793,-1.330636,0.428206,-0.697610,-0.651901,1.005021,0.152920,...,0.816570,-0.350184,-0.545396,0.490848,0.103133,1.401544,0.229745,0.749034,0.069906,-0.180338
4,-1.061733,0.687830,1.425894,-1.906622,-2.342140,1.650735,3.635573,2.565773,1.852576,0.609440,...,1.553783,-0.782880,0.069234,0.957521,0.234332,-1.111859,0.037335,-3.100852,1.112413,0.004360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,0.873236,0.740026,0.622424,-0.177617,0.185255,0.585632,-0.683913,-0.050856,-0.439472,0.757598,...,0.684746,-0.927112,-0.179648,0.101954,-0.456190,-0.609759,-0.515844,0.158775,-0.723930,-0.048411
751,1.400464,-2.112742,1.495668,1.204111,1.193576,0.798102,-0.518928,1.846648,0.932837,0.566077,...,1.047105,-0.790677,-0.738009,-0.209161,0.072059,1.549733,-0.491793,0.913824,-1.307352,0.090112
752,-1.040604,-1.014831,-1.057285,-0.373652,0.723299,0.776084,-0.716910,-1.308599,0.332655,0.488986,...,0.788456,-0.424249,-0.393903,-0.468424,-0.380232,-1.146145,1.744976,0.313230,0.219746,0.136287
753,-0.330100,0.338658,-0.920734,1.292643,-0.001218,1.014425,-0.654029,-1.136803,0.868764,0.640758,...,0.394235,-0.428147,-0.077931,-0.183235,0.493277,1.961757,-0.227229,0.433234,-0.669732,-0.008833


In [15]:
# # 适应 Boruta 模型
# boruta_selector.fit(X, y)

# # 获取被认为是重要的特征
# important_features = boruta_selector.support_
# # 获取选定的特征索引
# selected_indices = np.where(important_features)[0]

# # 打印被选中的特征的数量
# print(f"被选中的特征数量: {len(selected_indices)}")

# # 选择前 1000 个重要的 mRNA 特征（如果有的话）
# if len(selected_indices) > 1000:
#     selected_indices = selected_indices[:1000]

# # 打印前 1000 个重要的特征名称
# selected_features = [f'mRNA_{i + 1}' for i in selected_indices]
# print("前 1000 个重要的 mRNA 特征:", selected_features)

# # 创建数据框输出选择的特征
# # 确保 final_selected_indices 变量在之前的代码中正确获取
# Methylation = Methylation.iloc[:, selected_indices].copy()  # 使用选定的特征索引

# # 对数据框进行 Z 得分归一化
# Methylation = (Methylation - Methylation.mean()) / Methylation.std()

# # 输出结果
# print("选择的特征数据框：")
# Methylation

In [16]:
# 定义特征和响应变量
X = SCNA
y = Clinical['status']

In [17]:
# 分割数据集：80% 用于训练，20% 用于测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建随机森林分类器
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=None)

# 创建 Boruta 特征选择器
boruta_selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',  # 自动选择随机森林的树的数量
    perc=80,  # 设置80%阈值，选择比80%分位数更重要的特征
    verbose=2,  # 输出详细信息
    random_state=42
)

# 训练 Boruta 模型
boruta_selector.fit(X_train.values, y_train.values)

# 获取被认为是重要的特征
important_features = X.columns[boruta_selector.support_]

# 获取可能重要的特征
tentative_features = X.columns[boruta_selector.support_weak_]

# 合并重要特征和可能重要的特征
selected_features = list(important_features) + list(tentative_features)

# 创建一个新的数据框，包含所有被选中的特征和目标变量
X_selected = X[selected_features]

# 创建 StandardScaler 对象
scaler = StandardScaler()

# 对选择的特征进行 Z 得分归一化
X_selected_scaled = scaler.fit_transform(X_selected)

# 将归一化后的数据转换回 DataFrame，并保留原有的列名
SCNA = pd.DataFrame(X_selected_scaled, columns=X_selected.columns)

# 打印归一化后的特征数据框
print("Z 得分归一化后的特征数据框:")
SCNA

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	13239
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	4839
Rejected: 	8400
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	4839
Rejected: 	8400
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	4839
Rejected: 	8400
Iteration: 	11 / 100
Confirmed: 	0
Tentative: 	4839
Rejected: 	8400
Iteration: 	12 / 100
Confirmed: 	0
Tentative: 	2554
Rejected: 	10685
Iteration: 	13 / 100
Confirmed: 	17
Tentative: 	2537
Rejected: 	10685
Iteration: 	14 / 100
Confirmed: 	17
Tentative: 	2537
Rejected: 	10685
Iteration: 	15 / 100
Confirmed: 	28
Tentative: 	2526
Rejected: 	10

,MYC,EXT1,ZHX2,DSCAM,RASGEF1A,HAS2,TNFRSF11B,ADAM9,SLC30A8,IDO1,...,LCA5L,UTP23,GABRA3,CRAT,NUP188,MIR224,KLB,DYNLT3,RPL18A,HMGN1
0,-0.526486,-0.498247,-0.492539,0.203585,0.055309,-0.495001,-0.499462,-0.170461,-0.515628,-0.131567,...,0.169679,-0.514767,0.092054,0.142953,0.141626,0.084117,0.708004,0.172594,0.351928,0.175798
1,2.025871,3.834357,2.429915,4.446245,-2.485791,2.414399,3.843892,-0.347270,3.692934,-0.329944,...,4.343009,3.735797,7.407535,-1.609097,-1.612991,7.625313,-2.430496,-0.975709,0.579272,4.459829
2,-0.410521,-0.370707,-0.359759,-0.119610,0.014141,-0.362814,-0.371605,3.214308,-0.391740,3.228795,...,-0.148234,-0.389642,-0.036834,-0.087879,-0.089544,-0.048747,0.275282,0.016983,-0.044090,-0.150547
3,-0.611754,-0.592026,-0.590171,-0.013837,0.385368,-0.592197,-0.593474,3.214308,-0.606722,3.666103,...,-0.044190,-0.606771,-0.707938,-1.408539,-1.412139,-0.740558,-0.803260,-0.793268,-1.881175,-0.043743
4,3.290112,0.121947,-1.539155,0.086060,2.372821,-1.536941,-1.507270,0.266189,0.086810,0.358349,...,0.054074,0.093683,-2.152368,-2.025351,-2.029855,-2.229555,0.403577,1.438948,-2.178189,0.057127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,-0.243851,-0.177647,-0.158247,-0.126661,0.004203,-0.162720,-0.178066,-0.347270,-0.204207,-0.329944,...,-0.155171,-0.200238,-0.036834,0.165657,0.164364,-0.048747,0.298477,0.016983,-0.110093,-0.157668
751,3.412898,-2.107500,2.480683,0.083121,-1.236535,-2.158996,-2.087637,-0.288659,3.692934,-0.264184,...,0.051184,3.735797,-0.001279,1.331168,1.331583,-0.012095,0.265860,-0.253458,0.634274,0.054160
752,-0.195646,-0.750826,-0.755494,-0.078476,-0.054001,-0.756782,-0.752667,-0.766337,-0.760976,-0.308024,...,-0.107773,-0.762563,-0.307942,0.214851,0.213630,-0.328221,-1.480973,-0.288873,0.021913,-0.109012
753,-0.127431,-0.059360,-0.035621,-1.253728,-1.254280,-0.040124,-0.059486,-0.731170,-0.089306,-0.760675,...,-1.263820,-0.084191,-0.707938,-1.230685,-1.234024,-0.740558,0.468811,-0.782536,-1.470490,-1.295725


In [18]:
# # 定义特征和响应变量
# X = SCNA.values
# # 适应 Boruta 模型
# boruta_selector.fit(X, y)

# # 获取被认为是重要的特征
# important_features = boruta_selector.support_
# # 获取选定的特征索引
# selected_indices = np.where(important_features)[0]

# # 打印被选中的特征的数量
# print(f"被选中的特征数量: {len(selected_indices)}")

# # 选择前 1000 个重要的 mRNA 特征（如果有的话）
# if len(selected_indices) > 1000:
#     selected_indices = selected_indices[:1000]

# # 打印前 1000 个重要的特征名称
# selected_features = [f'mRNA_{i + 1}' for i in selected_indices]
# print("前 1000 个重要的 mRNA 特征:", selected_features)

# # 创建数据框输出选择的特征
# # 确保 final_selected_indices 变量在之前的代码中正确获取
# SCNA = SCNA.iloc[:, selected_indices].copy()  # 使用选定的特征索引

# # 对数据框进行 Z 得分归一化
# SCNA = (SCNA - SCNA.mean()) / SCNA.std()

# # 输出结果
# print("选择的特征数据框：")
# SCNA

In [19]:
# 定义特征和响应变量
X = RNAseq
y = Clinical['status']

In [20]:
# 分割数据集：80% 用于训练，20% 用于测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建随机森林分类器
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=None)

# 创建 Boruta 特征选择器
boruta_selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',  # 自动选择随机森林的树的数量
    perc=80,  # 设置80%阈值，选择比80%分位数更重要的特征
    verbose=2,  # 输出详细信息
    random_state=42
)

# 训练 Boruta 模型
boruta_selector.fit(X_train.values, y_train.values)

# 获取被认为是重要的特征
important_features = X.columns[boruta_selector.support_]

# 获取可能重要的特征
tentative_features = X.columns[boruta_selector.support_weak_]

# 合并重要特征和可能重要的特征
selected_features = list(important_features) + list(tentative_features)

# 创建一个新的数据框，包含所有被选中的特征和目标变量
X_selected = X[selected_features]

# 创建 StandardScaler 对象
scaler = StandardScaler()

# 对选择的特征进行 Z 得分归一化
X_selected_scaled = scaler.fit_transform(X_selected)

# 将归一化后的数据转换回 DataFrame，并保留原有的列名
RNAseq = pd.DataFrame(X_selected_scaled, columns=X_selected.columns)

# 打印归一化后的特征数据框
print("Z 得分归一化后的特征数据框:")
RNAseq

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	11094
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	6536
Rejected: 	4558
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	6536
Rejected: 	4558
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	6536
Rejected: 	4558
Iteration: 	11 / 100
Confirmed: 	84
Tentative: 	6452
Rejected: 	4558
Iteration: 	12 / 100
Confirmed: 	84
Tentative: 	4387
Rejected: 	6623
Iteration: 	13 / 100
Confirmed: 	84
Tentative: 	4387
Rejected: 	6623
Iteration: 	14 / 100
Confirmed: 	134
Tentative: 	4337
Rejected: 	6623
Iteration: 	15 / 100
Confirmed: 	134
Tentative: 	4337
Rejected: 	6

,ACAT1,CCDC40,CRIP1,FASN,FOXF1,MESP1,WNT11,ATP6V1B1,SIX2,STOX1,...,PLEKHA3,ME3,MRPS6,WDR73,OR2B6,MTO1,PGBD5,POP1,CA14,UQCRHL
0,0.845700,0.129842,-0.323975,0.506525,0.553363,-0.966502,-0.228337,0.386050,-0.245566,-0.099459,...,0.863041,0.466649,-0.561332,0.072800,-1.397575,-0.272751,-0.304549,-0.328754,-0.350449,-1.700401
1,-0.502270,-1.230730,0.744408,-1.921993,0.322220,2.207570,1.344383,2.543040,-0.269194,-0.559494,...,-0.406165,0.771866,3.511991,-0.227657,1.756033,-0.241166,2.760908,0.060754,-0.284240,2.619295
2,-0.161086,0.325768,-0.641586,1.666888,0.877235,-0.321958,0.859741,0.380148,-0.399323,-0.278608,...,-0.813311,0.441303,-0.306178,-0.157261,-0.440307,0.359611,-0.009952,-0.408155,-0.196760,-0.305205
3,-1.578447,0.791427,0.633325,2.177636,-1.132720,-1.157383,-1.604243,2.020307,-1.376559,-1.900296,...,-0.743710,-0.968390,0.864946,0.273984,0.623153,-0.536995,-0.438013,0.344312,2.131915,0.217647
4,-0.894981,0.366627,-3.055921,-1.793056,-1.800688,1.601545,0.285722,0.820591,0.026988,1.869489,...,-1.754525,1.359498,0.998432,1.289722,0.704698,-1.133546,1.460170,-0.661231,-0.069062,1.356574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,1.966947,0.656329,0.966965,-1.452533,0.313281,0.064845,0.738338,-1.680447,0.215352,1.654834,...,1.259042,1.271321,1.199530,-0.328438,-0.143196,1.147672,0.171891,-1.205731,0.076651,-0.235005
751,1.063491,2.396848,-0.124205,-0.168019,0.168274,-1.524355,-0.474213,1.765060,1.758641,0.365990,...,-0.091138,0.416941,0.510031,1.191395,1.144803,0.158759,0.547138,-0.454124,-1.705735,-1.104329
752,0.480058,-1.220745,-0.240649,1.886831,-1.207842,-0.659970,0.495070,-0.070842,0.397442,0.833759,...,-1.232968,0.719616,0.501678,0.321355,0.091320,-0.166653,-0.669245,-0.309703,-0.174424,0.559589
753,0.879777,0.227459,2.046015,1.128995,-1.116287,1.248078,-0.886208,0.167608,0.606803,-1.344218,...,0.539825,-0.949524,0.152752,1.095709,0.718620,-3.360937,0.254955,0.104757,1.201403,0.541347


In [21]:
# # 定义特征和响应变量
# X = RNAseq.values
# # 适应 Boruta 模型
# boruta_selector.fit(X, y)

# # 获取被认为是重要的特征
# important_features = boruta_selector.support_
# # 获取选定的特征索引
# selected_indices = np.where(important_features)[0]

# # 打印被选中的特征的数量
# print(f"被选中的特征数量: {len(selected_indices)}")

# # 选择前 1000 个重要的 mRNA 特征（如果有的话）
# if len(selected_indices) > 1000:
#     selected_indices = selected_indices[:1000]

# # 打印前 1000 个重要的特征名称
# selected_features = [f'mRNA_{i + 1}' for i in selected_indices]
# print("前 1000 个重要的 mRNA 特征:", selected_features)

# # 创建数据框输出选择的特征
# # 确保 final_selected_indices 变量在之前的代码中正确获取
# RNAseq = RNAseq.iloc[:, selected_indices].copy()  # 使用选定的特征索引

# # 对数据框进行 Z 得分归一化
# RNAseq = (RNAseq - RNAseq.mean()) / RNAseq.std()

# # 输出结果
# print("选择的特征数据框：")
# RNAseq

In [22]:
X = Methylation
y = Clinical.iloc[:,-2]
Methylation_X_train, Methylation_X_test, Methylation_y_train, Methylation_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Methylation_X_train, Methylation_X_val, Methylation_y_train, Methylation_y_val = train_test_split(Methylation_X_train, Methylation_y_train, test_size=0.25, random_state=42)

In [23]:
Methylation_X_train.to_csv('../../DATA/BRCA/Methylation_train.csv', index=False)

In [24]:
Methylation_X_test.to_csv('../../DATA/BRCA/Methylation_test.csv', index=False)

In [25]:
Methylation_X_val.to_csv('../../DATA/BRCA/Methylation_val.csv', index=False)

In [26]:
Methylation_y_train.to_csv('../../DATA/BRCA/labels_train.csv', index=False)

In [27]:
Methylation_y_test.to_csv('../../DATA/BRCA/labels_test.csv', index=False)

In [28]:
Methylation_y_val.to_csv('../../DATA/BRCA/labels_val.csv', index=False)

In [29]:
X = SCNA
y = Clinical.iloc[:,-2]
SCNA_X_train, SCNA_X_test, SCNA_y_train, SCNA_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
SCNA_X_train, SCNA_X_val, SCNA_y_train, SCNA_y_val = train_test_split(SCNA_X_train, SCNA_y_train, test_size=0.25, random_state=42)

In [30]:
SCNA_X_train.to_csv('../../DATA/BRCA/SCNA_train.csv', index=False)

In [31]:
SCNA_X_test.to_csv('../../DATA/BRCA/SCNA_test.csv', index=False)

In [32]:
SCNA_X_val.to_csv('../../DATA/BRCA/SCNA_val.csv', index=False)

In [37]:
X = RNAseq
y = Clinical.iloc[:,-2]
RNAseq_X_train, RNAseq_X_test, RNAseq_y_train, RNAseq_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
RNAseq_X_train, RNAseq_X_val, RNAseq_y_train, RNAseq_y_val = train_test_split(RNAseq_X_train, RNAseq_y_train, test_size=0.25, random_state=42)

In [38]:
RNAseq_X_train.to_csv('../../DATA/BRCA/RNAseq_train.csv', index=False)

In [39]:
RNAseq_X_test.to_csv('../../DATA/BRCA/RNAseq_test.csv', index=False)

In [40]:
RNAseq_X_val.to_csv('../../DATA/BRCA/RNAseq_val.csv', index=False)